In [1]:
import pandas as pd
import nltk
import re
import numpy as np
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train_data['len'] = train_data['tweet'].str.len()
test_data['len'] = test_data['tweet'].str.len()

In [3]:
nltk.download('stopwords')
train_corpus = []

for i in range(0, 31962):
    review = re.sub('[^a-zA-Z]', ' ', train_data['tweet'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
  # stemming
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  # joining them back with space
    review = ' '.join(review)
    train_corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
test_corpus = []

for i in range(0, 17197):
    review = re.sub('[^a-zA-Z]', ' ', test_data['tweet'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
  # stemming
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  # joining them back with space
    review = ' '.join(review)
    test_corpus.append(review)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 2500)
x = cv.fit_transform(train_corpus).toarray()
y = train_data.iloc[:, 1]

print(x.shape)
print(y.shape)

(31962, 2500)
(31962,)


In [6]:
# creating bag of words

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 2500)
x_test = cv.fit_transform(test_corpus).toarray()

print(x_test.shape)

(17197, 2500)


In [7]:
# splitting the training data into train and valid sets

from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.25, random_state = 42)

print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(23971, 2500)
(7991, 2500)
(23971,)
(7991,)


In [8]:
# standardization

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_valid = sc.transform(x_valid)
x_test = sc.transform(x_test)

In [10]:
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.svm import SVC

model = SVC()
model.fit(x_train, y_train)

y_pred = model.predict(x_valid)

print("Training Accuracy :", model.score(x_train, y_train))
print("Validation Accuracy :", model.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("f1 score :", f1_score(y_valid, y_pred))

# confusion matrix
cm = confusion_matrix(y_valid, y_pred)
print(cm)

Training Accuracy : 0.978181969880272
Validation Accuracy : 0.9521962207483419
f1 score : 0.4986876640419947
[[7419   13]
 [ 369  190]]


In [11]:
prediction = model.predict(x_test)
print(prediction)

[0 0 0 ... 0 0 0]


In [12]:
test_data['label'] = prediction
output = test_data[['id','label']]
output.to_csv('output.csv', index=False)